In [21]:
import os, json
import numpy as np
import torch.optim as optim
import torch.nn as nn
from qiskit.providers.fake_provider import FakeLima
from scripts.data_setup import load_data
from scripts.from_circ_to_numpy import operations_to_features, save_to_json, load_from_json
from scripts.model import create_models, train_and_test_step, save_models, load_models
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import BackendEstimator
from zne import zne, ZNEStrategy
from zne.noise_amplification import *
from zne.extrapolation import *
from qiskit_aer import AerSimulator
from sklearn.metrics import root_mean_squared_error
from zne.noise_amplification import LocalFoldingAmplifier, GlobalFoldingAmplifier
from zne.extrapolation import PolynomialExtrapolator
import qiskit
from qiskit_aer import QasmSimulator
#from qiskit.circuit.classical import *

In [22]:
dir_models = 'experiments/test_2'
dir_data = 'data_small_1'

n_qubits = 5 # 5 == n_qubits, default value for now because of FakeLima
train_circuits, train_observables, train_ideal_exp_vals, train_noisy_exp_vals, test_circuits, test_observables, test_ideal_exp_vals, test_noisy_exp_vals = load_data(f'data/circuits/{dir_data}')
X_train, y_train, X_test, y_test = load_from_json(f'data/features/{dir_data}')

In [23]:
circuit = test_circuits[0]

In [24]:
test_circuits[0]

In [25]:
backend = FakeLima()
backend_noisy = AerSimulator.from_backend(FakeLima())
#backend = QasmSimulator(noise_model=backend_noisy)
backend = backend_noisy

In [26]:
import numpy as np
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq

In [27]:
def ibmq_executor(circuit: qiskit.QuantumCircuit, shots: int = 8192) -> float:
    """Returns the expectation value to be mitigated.

    Args:
        circuit: Circuit to run.
        shots: Number of times to execute the circuit to compute the expectation value.
    """
    # Transpile the circuit so it can be properly run
    exec_circuit = qiskit.transpile(
        circuit,
        backend=backend,
        optimization_level=0, # Important to preserve folded gates.
    )
    
    # Run the circuit
    job = backend.run(circuit, shots=shots)

    # Convert from raw measurement counts to the expectation value
    counts = job.result().get_counts()
    if counts.get("0") is None:
        expectation_value = 0.
    else:
        expectation_value = counts.get("0") / shots
    return expectation_value

In [28]:
unmitigated = ibmq_executor(circuit)
mitigated = zne.execute_with_zne(circuit, ibmq_executor)
print(f"Unmitigated result {unmitigated:.3f}")
print(f"Mitigated result {mitigated:.3f}")

ERROR - Could not load 'ibm_backend': No module named 'qiskit.circuit.classical'
ERROR - Could not load 'ibm_dynamic_circuits': No module named 'qiskit.circuit.classical'
ERROR - Could not load 'clifford.rb_default': No module named 'qiskit.passmanager'
ERROR - Could not load 'clifford.rb_default': No module named 'qiskit.passmanager'


AerError: 'unknown instruction: xx_minus_yy'